In [1]:
import pandas as pd
import numpy as np
import joblib
import json
import re
import matplotlib.pyplot as plt

In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error

In [3]:
import xgboost as xgb

In [4]:
data = pd.read_csv('giraffe.csv')
df = pd.DataFrame(data)
df

,years_to_retire,location,salary,investment_amount,current_savings,debt,other_expenses,number_of_dependents,current_invested_amount,house_ownership,...,s3_high,s4_high,s5_high,s6_high,profit_low,profit_mid,profit_high,goal_low,goal_mid,goal_high
0,33,Lucknow,183246.000000,0.000000,1.199280e+05,159214.000000,134129.000000,4,0.0,Rent,...,11.333333,8.333333,1.333333,20.333333,-159.214000,-159.214000,-159.214000,-1.592140e+02,-1.592140e+02,-1.592140e+02
1,19,Kolkata,29717.000000,0.000000,1.082460e+05,167850.089575,37478.000000,3,27235.0,Rent,...,0.000000,7.333333,0.000000,9.333333,-73.254000,-73.254000,-73.254000,-7.325400e+01,-7.325400e+01,-7.325400e+01
2,12,Chennai,352267.480339,18725.000000,1.339139e+06,0.000000,114612.000000,2,691978.0,Own House,...,2.833333,0.000000,0.833333,19.833333,1973.077697,2621.500000,4070.459225,3.686374e+06,4.210035e+06,5.562288e+06
3,25,Pune,176368.000000,17375.000000,6.646230e+05,27724.000000,99438.000000,3,324632.0,Own House,...,5.666667,3.666667,2.666667,26.666667,1629.851000,2512.454667,3742.651000,1.034861e+07,1.407751e+07,2.738906e+07
4,23,Bangalore,480555.000000,35500.000000,1.463199e+06,0.000000,199024.000000,3,733991.0,Own House,...,17.000000,5.000000,0.000000,10.000000,3844.997111,5434.624000,8332.737500,1.832744e+07,2.422332e+07,4.419480e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,16,Delhi,154592.000000,21992.000000,1.391306e+06,0.000000,100070.000000,0,757185.0,EMI,...,17.000000,0.000000,1.000000,17.000000,2137.622400,3372.639362,4941.998256,6.451133e+06,7.750453e+06,1.143923e+07
49996,22,Lucknow,324110.000000,55595.000000,3.459572e+06,0.000000,171171.000000,4,1063342.0,Rent,...,0.000000,0.000000,0.000000,3.333333,5760.642710,8473.320431,15180.214750,2.665545e+07,3.473565e+07,6.140760e+07
49997,38,Hyderabad,414081.000000,159274.739308,1.232281e+04,134015.000000,202310.000000,4,0.0,Own House,...,0.000000,0.000000,3.000000,3.000000,6050.436042,8011.571800,13579.173205,7.881014e+07,1.311552e+08,3.995899e+08
49998,7,Delhi,137956.000000,12257.000000,4.906520e+05,0.000000,340969.010432,4,240204.0,EMI,...,8.500000,5.500000,4.500000,11.500000,1181.574800,1637.535200,2647.512000,1.230910e+06,1.325381e+06,1.543290e+06


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 34 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   years_to_retire          50000 non-null  int64  
 1   location                 50000 non-null  object 
 2   salary                   50000 non-null  float64
 3   investment_amount        50000 non-null  float64
 4   current_savings          50000 non-null  float64
 5   debt                     50000 non-null  float64
 6   other_expenses           50000 non-null  float64
 7   number_of_dependents     50000 non-null  int64  
 8   current_invested_amount  50000 non-null  float64
 9   house_ownership          50000 non-null  object 
 10  s1_low                   50000 non-null  float64
 11  s2_low                   50000 non-null  float64
 12  s3_low                   50000 non-null  float64
 13  s4_low                   50000 non-null  float64
 14  s5_low                

In [6]:
data = data[(data['investment_amount'] != 0) &
            (data['years_to_retire'] != 0) &
            (data['salary'] != 0)]

In [7]:
data

,years_to_retire,location,salary,investment_amount,current_savings,debt,other_expenses,number_of_dependents,current_invested_amount,house_ownership,...,s3_high,s4_high,s5_high,s6_high,profit_low,profit_mid,profit_high,goal_low,goal_mid,goal_high
2,12,Chennai,352267.480339,18725.000000,1.339139e+06,0.0,114612.000000,2,691978.0,Own House,...,2.833333,0.000000,0.833333,19.833333,1973.077697,2621.500000,4070.459225,3.686374e+06,4.210035e+06,5.562288e+06
3,25,Pune,176368.000000,17375.000000,6.646230e+05,27724.0,99438.000000,3,324632.0,Own House,...,5.666667,3.666667,2.666667,26.666667,1629.851000,2512.454667,3742.651000,1.034861e+07,1.407751e+07,2.738906e+07
4,23,Bangalore,480555.000000,35500.000000,1.463199e+06,0.0,199024.000000,3,733991.0,Own House,...,17.000000,5.000000,0.000000,10.000000,3844.997111,5434.624000,8332.737500,1.832744e+07,2.422332e+07,4.419480e+07
6,15,Hyderabad,294754.000000,40682.771633,2.281321e+06,0.0,112682.000000,2,1397582.0,Own House,...,7.500000,2.500000,0.000000,9.500000,1530.998640,2042.058200,3658.277616,3.962486e+06,4.699106e+06,6.741703e+06
7,15,Mumbai,420437.981495,9641.000000,1.236218e+06,0.0,131112.000000,3,1000458.0,Own House,...,3.500000,5.500000,0.000000,19.500000,1149.457866,1370.950200,2087.662140,2.578082e+06,3.057204e+06,4.385750e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,16,Delhi,154592.000000,21992.000000,1.391306e+06,0.0,100070.000000,0,757185.0,EMI,...,17.000000,0.000000,1.000000,17.000000,2137.622400,3372.639362,4941.998256,6.451133e+06,7.750453e+06,1.143923e+07
49996,22,Lucknow,324110.000000,55595.000000,3.459572e+06,0.0,171171.000000,4,1063342.0,Rent,...,0.000000,0.000000,0.000000,3.333333,5760.642710,8473.320431,15180.214750,2.665545e+07,3.473565e+07,6.140760e+07
49997,38,Hyderabad,414081.000000,159274.739308,1.232281e+04,134015.0,202310.000000,4,0.0,Own House,...,0.000000,0.000000,3.000000,3.000000,6050.436042,8011.571800,13579.173205,7.881014e+07,1.311552e+08,3.995899e+08
49998,7,Delhi,137956.000000,12257.000000,4.906520e+05,0.0,340969.010432,4,240204.0,EMI,...,8.500000,5.500000,4.500000,11.500000,1181.574800,1637.535200,2647.512000,1.230910e+06,1.325381e+06,1.543290e+06


In [8]:
data['goal_low'] = data['goal_low'].map('{:,.2f}'.format)
data['goal_mid'] = data['goal_mid'].map('{:,.2f}'.format)
data['goal_high'] = data['goal_high'].map('{:,.2f}'.format)
data

,years_to_retire,location,salary,investment_amount,current_savings,debt,other_expenses,number_of_dependents,current_invested_amount,house_ownership,...,s3_high,s4_high,s5_high,s6_high,profit_low,profit_mid,profit_high,goal_low,goal_mid,goal_high
2,12,Chennai,352267.480339,18725.000000,1.339139e+06,0.0,114612.000000,2,691978.0,Own House,...,2.833333,0.000000,0.833333,19.833333,1973.077697,2621.500000,4070.459225,"3,686,373.92","4,210,035.09","5,562,287.82"
3,25,Pune,176368.000000,17375.000000,6.646230e+05,27724.0,99438.000000,3,324632.0,Own House,...,5.666667,3.666667,2.666667,26.666667,1629.851000,2512.454667,3742.651000,"10,348,611.04","14,077,508.12","27,389,058.75"
4,23,Bangalore,480555.000000,35500.000000,1.463199e+06,0.0,199024.000000,3,733991.0,Own House,...,17.000000,5.000000,0.000000,10.000000,3844.997111,5434.624000,8332.737500,"18,327,437.23","24,223,318.39","44,194,802.71"
6,15,Hyderabad,294754.000000,40682.771633,2.281321e+06,0.0,112682.000000,2,1397582.0,Own House,...,7.500000,2.500000,0.000000,9.500000,1530.998640,2042.058200,3658.277616,"3,962,485.86","4,699,106.33","6,741,703.09"
7,15,Mumbai,420437.981495,9641.000000,1.236218e+06,0.0,131112.000000,3,1000458.0,Own House,...,3.500000,5.500000,0.000000,19.500000,1149.457866,1370.950200,2087.662140,"2,578,082.16","3,057,204.45","4,385,749.85"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,16,Delhi,154592.000000,21992.000000,1.391306e+06,0.0,100070.000000,0,757185.0,EMI,...,17.000000,0.000000,1.000000,17.000000,2137.622400,3372.639362,4941.998256,"6,451,133.48","7,750,452.96","11,439,233.68"
49996,22,Lucknow,324110.000000,55595.000000,3.459572e+06,0.0,171171.000000,4,1063342.0,Rent,...,0.000000,0.000000,0.000000,3.333333,5760.642710,8473.320431,15180.214750,"26,655,446.89","34,735,647.66","61,407,598.19"
49997,38,Hyderabad,414081.000000,159274.739308,1.232281e+04,134015.0,202310.000000,4,0.0,Own House,...,0.000000,0.000000,3.000000,3.000000,6050.436042,8011.571800,13579.173205,"78,810,143.52","131,155,231.39","399,589,922.46"
49998,7,Delhi,137956.000000,12257.000000,4.906520e+05,0.0,340969.010432,4,240204.0,EMI,...,8.500000,5.500000,4.500000,11.500000,1181.574800,1637.535200,2647.512000,"1,230,909.87","1,325,381.04","1,543,290.39"


In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 37901 entries, 2 to 49999
Data columns (total 34 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   years_to_retire          37901 non-null  int64  
 1   location                 37901 non-null  object 
 2   salary                   37901 non-null  float64
 3   investment_amount        37901 non-null  float64
 4   current_savings          37901 non-null  float64
 5   debt                     37901 non-null  float64
 6   other_expenses           37901 non-null  float64
 7   number_of_dependents     37901 non-null  int64  
 8   current_invested_amount  37901 non-null  float64
 9   house_ownership          37901 non-null  object 
 10  s1_low                   37901 non-null  float64
 11  s2_low                   37901 non-null  float64
 12  s3_low                   37901 non-null  float64
 13  s4_low                   37901 non-null  float64
 14  s5_low                   37

In [10]:
data['goal_low'] = data['goal_low'].replace(',','',regex=True)
data['goal_mid'] = data['goal_mid'].replace(',','',regex=True)
data['goal_high'] = data['goal_high'].replace(',','',regex=True)
data

,years_to_retire,location,salary,investment_amount,current_savings,debt,other_expenses,number_of_dependents,current_invested_amount,house_ownership,...,s3_high,s4_high,s5_high,s6_high,profit_low,profit_mid,profit_high,goal_low,goal_mid,goal_high
2,12,Chennai,352267.480339,18725.000000,1.339139e+06,0.0,114612.000000,2,691978.0,Own House,...,2.833333,0.000000,0.833333,19.833333,1973.077697,2621.500000,4070.459225,3686373.92,4210035.09,5562287.82
3,25,Pune,176368.000000,17375.000000,6.646230e+05,27724.0,99438.000000,3,324632.0,Own House,...,5.666667,3.666667,2.666667,26.666667,1629.851000,2512.454667,3742.651000,10348611.04,14077508.12,27389058.75
4,23,Bangalore,480555.000000,35500.000000,1.463199e+06,0.0,199024.000000,3,733991.0,Own House,...,17.000000,5.000000,0.000000,10.000000,3844.997111,5434.624000,8332.737500,18327437.23,24223318.39,44194802.71
6,15,Hyderabad,294754.000000,40682.771633,2.281321e+06,0.0,112682.000000,2,1397582.0,Own House,...,7.500000,2.500000,0.000000,9.500000,1530.998640,2042.058200,3658.277616,3962485.86,4699106.33,6741703.09
7,15,Mumbai,420437.981495,9641.000000,1.236218e+06,0.0,131112.000000,3,1000458.0,Own House,...,3.500000,5.500000,0.000000,19.500000,1149.457866,1370.950200,2087.662140,2578082.16,3057204.45,4385749.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,16,Delhi,154592.000000,21992.000000,1.391306e+06,0.0,100070.000000,0,757185.0,EMI,...,17.000000,0.000000,1.000000,17.000000,2137.622400,3372.639362,4941.998256,6451133.48,7750452.96,11439233.68
49996,22,Lucknow,324110.000000,55595.000000,3.459572e+06,0.0,171171.000000,4,1063342.0,Rent,...,0.000000,0.000000,0.000000,3.333333,5760.642710,8473.320431,15180.214750,26655446.89,34735647.66,61407598.19
49997,38,Hyderabad,414081.000000,159274.739308,1.232281e+04,134015.0,202310.000000,4,0.0,Own House,...,0.000000,0.000000,3.000000,3.000000,6050.436042,8011.571800,13579.173205,78810143.52,131155231.39,399589922.46
49998,7,Delhi,137956.000000,12257.000000,4.906520e+05,0.0,340969.010432,4,240204.0,EMI,...,8.500000,5.500000,4.500000,11.500000,1181.574800,1637.535200,2647.512000,1230909.87,1325381.04,1543290.39


In [ ]:
data['profit_high'].max()

In [11]:
data['goal_low'] = pd.to_numeric(data['goal_low'])
data['goal_mid'] = pd.to_numeric(data['goal_mid'])
data['goal_high'] = pd.to_numeric(data['goal_high'])
data

,years_to_retire,location,salary,investment_amount,current_savings,debt,other_expenses,number_of_dependents,current_invested_amount,house_ownership,...,s3_high,s4_high,s5_high,s6_high,profit_low,profit_mid,profit_high,goal_low,goal_mid,goal_high
2,12,Chennai,352267.480339,18725.000000,1.339139e+06,0.0,114612.000000,2,691978.0,Own House,...,2.833333,0.000000,0.833333,19.833333,1973.077697,2621.500000,4070.459225,3686373.92,4.210035e+06,5.562288e+06
3,25,Pune,176368.000000,17375.000000,6.646230e+05,27724.0,99438.000000,3,324632.0,Own House,...,5.666667,3.666667,2.666667,26.666667,1629.851000,2512.454667,3742.651000,10348611.04,1.407751e+07,2.738906e+07
4,23,Bangalore,480555.000000,35500.000000,1.463199e+06,0.0,199024.000000,3,733991.0,Own House,...,17.000000,5.000000,0.000000,10.000000,3844.997111,5434.624000,8332.737500,18327437.23,2.422332e+07,4.419480e+07
6,15,Hyderabad,294754.000000,40682.771633,2.281321e+06,0.0,112682.000000,2,1397582.0,Own House,...,7.500000,2.500000,0.000000,9.500000,1530.998640,2042.058200,3658.277616,3962485.86,4.699106e+06,6.741703e+06
7,15,Mumbai,420437.981495,9641.000000,1.236218e+06,0.0,131112.000000,3,1000458.0,Own House,...,3.500000,5.500000,0.000000,19.500000,1149.457866,1370.950200,2087.662140,2578082.16,3.057204e+06,4.385750e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,16,Delhi,154592.000000,21992.000000,1.391306e+06,0.0,100070.000000,0,757185.0,EMI,...,17.000000,0.000000,1.000000,17.000000,2137.622400,3372.639362,4941.998256,6451133.48,7.750453e+06,1.143923e+07
49996,22,Lucknow,324110.000000,55595.000000,3.459572e+06,0.0,171171.000000,4,1063342.0,Rent,...,0.000000,0.000000,0.000000,3.333333,5760.642710,8473.320431,15180.214750,26655446.89,3.473565e+07,6.140760e+07
49997,38,Hyderabad,414081.000000,159274.739308,1.232281e+04,134015.0,202310.000000,4,0.0,Own House,...,0.000000,0.000000,3.000000,3.000000,6050.436042,8011.571800,13579.173205,78810143.52,1.311552e+08,3.995899e+08
49998,7,Delhi,137956.000000,12257.000000,4.906520e+05,0.0,340969.010432,4,240204.0,EMI,...,8.500000,5.500000,4.500000,11.500000,1181.574800,1637.535200,2647.512000,1230909.87,1.325381e+06,1.543290e+06


In [12]:
pd.options.display.float_format = '{:,.2f}'.format

In [13]:
data

,years_to_retire,location,salary,investment_amount,current_savings,debt,other_expenses,number_of_dependents,current_invested_amount,house_ownership,...,s3_high,s4_high,s5_high,s6_high,profit_low,profit_mid,profit_high,goal_low,goal_mid,goal_high
2,12,Chennai,"352,267.48","18,725.00","1,339,139.00",0.00,"114,612.00",2,"691,978.00",Own House,...,2.83,0.00,0.83,19.83,"1,973.08","2,621.50","4,070.46","3,686,373.92","4,210,035.09","5,562,287.82"
3,25,Pune,"176,368.00","17,375.00","664,623.00","27,724.00","99,438.00",3,"324,632.00",Own House,...,5.67,3.67,2.67,26.67,"1,629.85","2,512.45","3,742.65","10,348,611.04","14,077,508.12","27,389,058.75"
4,23,Bangalore,"480,555.00","35,500.00","1,463,199.00",0.00,"199,024.00",3,"733,991.00",Own House,...,17.00,5.00,0.00,10.00,"3,845.00","5,434.62","8,332.74","18,327,437.23","24,223,318.39","44,194,802.71"
6,15,Hyderabad,"294,754.00","40,682.77","2,281,321.00",0.00,"112,682.00",2,"1,397,582.00",Own House,...,7.50,2.50,0.00,9.50,"1,531.00","2,042.06","3,658.28","3,962,485.86","4,699,106.33","6,741,703.09"
7,15,Mumbai,"420,437.98","9,641.00","1,236,218.00",0.00,"131,112.00",3,"1,000,458.00",Own House,...,3.50,5.50,0.00,19.50,"1,149.46","1,370.95","2,087.66","2,578,082.16","3,057,204.45","4,385,749.85"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,16,Delhi,"154,592.00","21,992.00","1,391,306.00",0.00,"100,070.00",0,"757,185.00",EMI,...,17.00,0.00,1.00,17.00,"2,137.62","3,372.64","4,942.00","6,451,133.48","7,750,452.96","11,439,233.68"
49996,22,Lucknow,"324,110.00","55,595.00","3,459,572.24",0.00,"171,171.00",4,"1,063,342.00",Rent,...,0.00,0.00,0.00,3.33,"5,760.64","8,473.32","15,180.21","26,655,446.89","34,735,647.66","61,407,598.19"
49997,38,Hyderabad,"414,081.00","159,274.74","12,322.81","134,015.00","202,310.00",4,0.00,Own House,...,0.00,0.00,3.00,3.00,"6,050.44","8,011.57","13,579.17","78,810,143.52","131,155,231.39","399,589,922.46"
49998,7,Delhi,"137,956.00","12,257.00","490,652.00",0.00,"340,969.01",4,"240,204.00",EMI,...,8.50,5.50,4.50,11.50,"1,181.57","1,637.54","2,647.51","1,230,909.87","1,325,381.04","1,543,290.39"


In [51]:
features_low = ['years_to_retire', 'investment_amount', 'debt', 'profit_low', 'current_savings']
target_low = 'goal_low'

In [52]:
features_mid = ['years_to_retire', 'investment_amount', 'debt', 'profit_mid', 'current_savings']
target_mid = 'goal_mid'

In [53]:
features_high = ['years_to_retire', 'investment_amount', 'debt', 'profit_high', 'current_savings']
target_high = 'goal_high'

In [54]:
data[target_mid] = np.log1p(data[target_mid])

In [55]:
x_mid = data[features_mid]

In [56]:
y_mid = data[target_low]

In [57]:
scaler = StandardScaler()

In [58]:
x_mid_scaled = scaler.fit_transform(x_mid)

In [59]:
x_mid_train, x_mid_test, y_mid_train, y_mid_test = train_test_split(x_mid_scaled, y_mid, test_size=0.3, random_state=42)

In [60]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Assuming data is your DataFrame containing all the relevant data
# Setup features based on the columns in your DataFrame
features_low = ['years_to_retire', 'investment_amount', 'debt', 'profit_low', 'current_savings']
features_high = ['years_to_retire', 'investment_amount', 'debt', 'profit_high', 'current_savings']

# Target columns, assuming they are already defined in your DataFrame
target_low = 'goal_low'  # Adjust if the column name is different
target_high = 'goal_high'  # Adjust if the column name is different

# Prepare low data
x_low = data[features_low]
y_low = np.log1p(data[target_low])  # Applying log1p transformation

# Prepare high data
x_high = data[features_high]
y_high = np.log1p(data[target_high])  # Applying log1p transformation

# Scaling features for low data
scaler_low = StandardScaler()
x_low_scaled = scaler_low.fit_transform(x_low)

# Scaling features for high data
scaler_high = StandardScaler()
x_high_scaled = scaler_high.fit_transform(x_high)

# Splitting the datasets into training and testing sets
x_low_train, x_low_test, y_low_train, y_low_test = train_test_split(x_low_scaled, y_low, test_size=0.3, random_state=42)
x_high_train, x_high_test, y_high_train, y_high_test = train_test_split(x_high_scaled, y_high, test_size=0.3, random_state=42)

# You can now proceed to train your model similar to how you trained the mid model


In [61]:
def train_xgboost(x_train, y_train, x_test, y_test):
    model = xgb.XGBRegressor(objective='reg:squarederror')

    param_grid = {
        'n_estimators': [100, 200],
        'learning_rate': [0.01, 0.1],
        'max_depth': [3, 7, 11],
        'min_child_weight': [1, 3, 5]
    }

    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error', n_jobs=-1)
    grid_search.fit(x_train, y_train)

    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(x_test)
    mse = mean_squared_error(y_test, y_pred)

    return best_model, mse

In [62]:
output_low = train_xgboost(x_low_train, y_low_train, x_low_test, y_low_test)

In [63]:
output_mid = train_xgboost(x_mid_train, y_mid_train, x_mid_test, y_mid_test)

In [64]:
output_high = train_xgboost(x_high_train, y_high_train, x_high_test, y_high_test)

In [66]:
model_low = output_low[0]
model_mid = output_mid[0]
model_high = output_high[0]

In [44]:
model_mid

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=7, max_leaves=None,
             min_child_weight=3, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=200, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [70]:
output_mid[1]

0.006382626581866058

In [74]:
y_high_pred = model_high.predict(x_high_test)
y_high_pred = np.expm1(y_high_pred)

In [77]:
y_high_test = np.log1p(y_high_test)

In [78]:
# Displaying a few predictions
predictions = pd.DataFrame({
    'goal_low_actual': y_high_test,
    'goal_low_predicted': y_high_pred
}).head()

predictions

,goal_low_actual,goal_low_predicted
36337,inf,"14,934,664.00"
48054,inf,"428,874.78"
1599,inf,"68,019,368.00"
26048,inf,"136,066,064.00"
16051,inf,"18,048,624.00"


'years_to_retire', 'investment_amount', 'debt', 'profit_low', 'current_savings'

In [79]:
joblib.dump(model_high, 'new_model_high.pkl')

['new_model_high.pkl']

In [31]:
joblib.dump(scaler, 'scaler_for_goal.pkl')

['scaler_for_goal.pkl']

In [80]:
async def goal_low(years_to_retire, investment_amount, debt, profit_low, current_savings):
    scaler_fun = joblib.load('scaler_for_goal.pkl')
    model_low_fun = joblib.load('new_model_high.pkl')
    input_fun = [years_to_retire, investment_amount, debt, profit_low, current_savings]
    scaled_input_fun = scaler_fun.fit_transform([input_fun])
    answer_fun = model_low_fun.predict(scaled_input_fun)
    output_fun = np.expm1(answer_fun)
    return output_fun

In [81]:
answer = await goal_low(15, 30000, 0, 15000, 0)
print(answer)

[24223096.]


In [82]:
type(answer)

numpy.ndarray

In [83]:
finale = answer[0]
finale

24223096.0

In [88]:
type(finale)

float

In [87]:
finale = float(finale)

# <strong style="color: yellow">Final Flow</strong>

# Load
``` bash
scaler_fun = joblib.load('scaler_for_goal.pkl')
model_low_fun = joblib.load('new_model_low.pkl')
model_low_fun = joblib.load('new_model_mid.pkl')
model_low_fun = joblib.load('new_model_high.pkl')
```

# Define Input array
``` bash
input = [years_to_retire, investment_amount, debt, profit, current_savings] # NOTE: Keep the profit according to Risk
```

# Scale Input
``` bash
scaled_input = scaler_fun.fit_transform([input])
```

# Predict
``` bash
answer = model.predict(scaled_input)
```

# Bring the output to desired form
``` bash
output = np.expm1(answer)
```

# Then take the Float value from the numpy array
``` bash 
final_answer = output[0]
```

# From numpy.float to float
``` bash
finale = float(final_answer)
```